## Connect to database

In [ ]:
from pymongo import MongoClient

def connect_to_prod_mongo():
    #mongo_host = "5.9.1.218"
    mongo_host = "138.201.51.230"
    mongo_port = 27017
    #mongo_db = "dataroot-master"
    mongo_db = "dataroot_nlp"
    #mongo_user = "new-main-worker"
    #mongo_password = "VX4xB,.i(12r{2X|N7~y}02]9533xbpA"
    connection = MongoClient(mongo_host, mongo_port)
    db = connection[mongo_db]
    #db.authenticate(mongo_user, mongo_password)
    return db

In [ ]:
db = connect_to_prod_mongo()

## Play to make out

In [ ]:
db.collection_names(include_system_collections=False)

In [ ]:
a = db.parsedHtml.find_one()

In [ ]:
[i['body']['link'] for i in a['pages']]

In [ ]:
t=[i['body']['text'] for i in a['pages']]

In [ ]:
a['pages'][0]['body']['text']

In [ ]:
#print used linkes
c=0
for doc in db.parsedHtml.find():
    for page in doc['pages']:
        c+=1
        print page['body']['link']
print c

## Preprocess preprocessing

In [ ]:
import numpy as np
import nltk
from nltk.metrics import *
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
def removePunctuation(text):
    p = re.compile('[^a-zA-Z0-9_ ]')
    return p.sub('', text.lower()).strip()

In [ ]:
with open('terrier-stop.txt') as f:
    stop_words = [i.strip() for i in f.readlines()]
print "We have %d stop words" % len(stop_words)

## Create one of datasets

In [ ]:
#site - words
documents = []
for doc in db.parsedHtml.find():
    documents += [filter(lambda x: x!= u'', 
                         map(lambda x: removePunctuation(x), #x.strip(), 
                             sum([i['body']['text'].split('\n') for i in doc['pages']],[])
                            )
                     )
                 ]

print "sites - %d" % len(documents)

In [ ]:
#page - words
documents = []
for doc in db.parsedHtml.find():
    for page in doc['pages']:
        documents += [filter(lambda x: x!= u'', 
                             map(lambda x: removePunctuation(x), #x.strip(), 
                                 page['body']['text'].split('\n')
                                )
                          )
                     ]
        
print "pages - %d" % len(documents)

In [ ]:
#sentence - words
documents = []
for doc in db.parsedHtml.find():
    for page in doc['pages']:
        documents += filter(lambda x: x!= u'',
                            map(lambda x: [removePunctuation(x)], #x.strip(), 
                                page['body']['text'].split('\n')
                               )
                        )
        
print "sentences - %d" % len(documents)

## Clean it (to be done)

In [ ]:
#filter code
for i, doc in enumerate(documents):
    documents[i] = [ d for d in doc if not('{' in d or '}' in d or d[-2:]==');')]

In [ ]:
documents = [doc for doc in documents if len(doc) >10]

## Save it file

In [ ]:
#with open('site - words.txt', 'w+') as f:
#with open('page - words.txt', 'w+') as f:
with open('sentence - words.txt', 'w+') as f:
    for doc in documents:
        f.write(' '.join(doc).encode('utf-8')+'\n')

## Just get rid of stopwords

In [ ]:
#cut out stop words ans stem
ps = PorterStemmer()
for i, el in enumerate(documents):
    for j, doc in enumerate(el):
        documents[i][j] = ' '.join(filter(lambda x: x not in stop_words, word_tokenize(doc)))
    documents[i] = filter(lambda x: x!=u'', documents[i])

## Just stem

In [ ]:
#cut out stop words ans stem
ps = PorterStemmer()
for i, el in enumerate(documents):
    for j, doc in enumerate(el):
        documents[i][j] = ' '.join(map(lambda x: ps.stem(x), word_tokenize(doc)))
    documents[i] = filter(lambda x: x!=u'', documents[i])

## Get rid of stopwords + stem

In [ ]:
#cut out stop words ans stem
ps = PorterStemmer()
for i, el in enumerate(documents):
    for j, doc in enumerate(el):
        documents[i][j] = ' '.join(
                                    map(lambda x: ps.stem(x),
                                        filter(lambda x: x not in stop_words, 
                                          word_tokenize(doc))
                                       )
                                   )
    documents[i] = filter(lambda x: x!=u'', documents[i])